In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

from lib.utils.factory import get_CATARACTS_data

In [ ]:
class Args:
    data_path = '/media/yannik/samsung_data_ssd_2/CATARACTS-videos-processed/'

args = Args()

data_conf = {
    'CROP_DIM': None,
    'SHAPE': '(3, 128, 128)',
    'NORM': '(.5, .5)',
    'RANDOM_BRIGHTNESS_CONTRAST': True,
    'RANDOM_H_FLIP': True,
    'PHASE_WEIGHTED_SAMPLING': True,
    'TOOL_WEIGHTED_SAMPLING': False,
    'FOLD': -1,
    'FRAME_STEP': 5,
    'N_SEQ_FRAMES': 5,
    'OVERLAPPING_CHUNKS': False,
    'REMOVE_IDLE': False  # Removes idle frame from training datset (only when using weighted sampling)
}
train_conf = {
    'BATCH_SIZE': 8,
    'NUM_WORKERS': 1,
    'VAL_SAMPLES': 8
}
train_ds, train_dl, val_ds, val_dl = get_CATARACTS_data(args, data_conf, train_conf)

ds = val_ds

In [ ]:
print(str(len(train_ds)) + " train samples")
print(str(len(val_ds)) + " val samples")

In [ ]:
ds[0][0].shape

In [ ]:
SHAPE = (1080, 1920)

fig, ax = plt.subplots(nrows=train_conf['VAL_SAMPLES'], ncols=data_conf['N_SEQ_FRAMES'], figsize=(data_conf['N_SEQ_FRAMES']*5, train_conf['VAL_SAMPLES']*5))
img, name, file_name, phase_label, tool_label = next(iter(train_dl))
N, T, C, H, W = img.shape
for n in range(N):
    for t in range(T):
        phase_label_name = ds.phase_label_names[phase_label[n, t].item()]
        tool_label_names = np.array(ds.tool_label_names).squeeze()[tool_label[n, t].numpy() > 0]
        _img = F.interpolate(img[n, t].unsqueeze(0), size=SHAPE)
        ax[n, t].imshow(.5 + .5*_img.squeeze(0).permute(1, 2, 0))
        ax[n, t].axis('off')
        ax[n, t].set_title(f'{file_name[n][t]}\n{phase_label_name}\n{tool_label_names}')
plt.show()